In [1]:
"""
This transformation brings historical data into each row and also provides target for each row. 
Applies binary encoding to Company column.
Removes normal date and preservers only julian.
Splits dataset into train and test part, because another transformations down in pipeline would created dataleaks otherwise.
"""

'\nThis transformation brings historical data into each row and also provides target for each row. \nApplies binary encoding to Company column.\nRemoves normal date and preservers only julian.\nSplits dataset into train and test part, because another transformations down in pipeline would created dataleaks otherwise.\n'

In [2]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv("stock_details_5_years.csv")
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
0,2018-11-29 00:00:00-05:00,43.829761,43.863354,42.639594,43.083508,167080000,0.00,0.0,AAPL
1,2018-11-29 00:00:00-05:00,104.769074,105.519257,103.534595,104.636131,28123200,0.00,0.0,MSFT
2,2018-11-29 00:00:00-05:00,54.176498,55.007500,54.099998,54.729000,31004000,0.00,0.0,GOOGL
3,2018-11-29 00:00:00-05:00,83.749496,84.499496,82.616501,83.678497,132264000,0.00,0.0,AMZN
4,2018-11-29 00:00:00-05:00,39.692784,40.064904,38.735195,39.037853,54917200,0.04,0.0,NVDA
...,...,...,...,...,...,...,...,...,...
602957,2023-11-29 00:00:00-05:00,26.360001,26.397499,26.120001,26.150000,1729147,0.00,0.0,PPL
602958,2023-11-29 00:00:00-05:00,27.680000,28.535000,27.680000,28.350000,1940066,0.00,0.0,FITB
602959,2023-11-29 00:00:00-05:00,75.940002,76.555000,75.257500,75.610001,298699,0.00,0.0,IFF
602960,2023-11-29 00:00:00-05:00,45.230000,45.259998,44.040001,44.209999,2217579,0.00,0.0,CCJ


In [4]:
def add_julian_date_to_data(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df['julian_date'] = df['Date'].apply(lambda x: x.to_julian_date())
    return df


In [5]:
def add_days_distance_between_target_and_last_data_to_data(df):
    """
    data have gaps (weekends etc.) - this will add the information about how long was the gap from last known data
    so on normal days it should be 1 but for example on monday it should be 3
    """
    df = df.copy()
    
    df["data_freshness"] = df["julian_date"] - df.shift(1)["julian_date"]
    
    return df


In [6]:
def add_targed_to_data(df):
    df = df.copy()
    df["target"] = df.shift(-1)["Close"]
    df["binary_target"] = (df["target"] > df["Close"]).astype(int)
    return df
    

In [7]:
def add_shifts_to_data(df):
    df = df.copy()
    window_size = 10
    columns_to_take_shifts = ["Open", "High", "Low", "Close", "Volume"]
    for i in range(window_size):
        for column in columns_to_take_shifts:
            # Calculate the ratio of the current value in 'column' to the value 'i+1' positions back.
            # If the denominator is 0, set the result to 'df[f"{column}"]/100' to handle division by zero.
            df[f"shift_{i}_{column}"] = np.where(df[f"{column}"].shift(i+1) == 0, df[f"{column}"]/100, df[f"{column}"] / df[f"{column}"].shift(i+1))
    return df
    
    
    
    

In [8]:
def split_by_company_apply_transform_merge_again(df):
    companies = df['Company'].unique()
    companies_dfs = []
    for company in companies:
        single_company_df =  df[df['Company'] == company]
        single_company_df = single_company_df.copy()
        single_company_df = add_julian_date_to_data(single_company_df)
        single_company_df = add_shifts_to_data(single_company_df)
        single_company_df = add_days_distance_between_target_and_last_data_to_data(single_company_df)
        single_company_df = add_targed_to_data(single_company_df)
        single_company_df = single_company_df.dropna()
        
        companies_dfs.append(single_company_df)
        
    combined_df = pd.concat(companies_dfs)
    sorted_df = combined_df.sort_values(by='Date')
    return sorted_df
        
    

In [ ]:
df = split_by_company_apply_transform_merge_again(df)
df

/tmp/ipykernel_2028/2502543417.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/tmp/ipykernel_2028/2502543417.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/tmp/ipykernel_2028/2502543417.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to t

In [ ]:
max(df["data_freshness"])

In [ ]:
def remove_company_fill_binary_encoding(df):
    df = df.copy()
    binary_encoder = ce.BinaryEncoder(cols=['Company'])
    df = binary_encoder.fit_transform(df)
    return df

In [ ]:
df = remove_company_fill_binary_encoding(df)

In [ ]:
def remove_date(df):
    df = df.copy()
    del df["Date"]
    return df

df = remove_date(df)

In [ ]:
df

In [ ]:
def split_into_train_and_test(df):
    df = df.copy()
    split_ratio = 0.8
    train_size = int(len(df) * split_ratio)
    train_df = df[:train_size]
    test_df = df[train_size:]
    return train_df.copy(), test_df.copy()


In [ ]:
train_df, test_df = split_into_train_and_test(df)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
def scale_dfs(train_df, test_df):
    columns_to_exclude=["target", "binary_target"]
    
    # Create copies of the input DataFrames
    train_df = train_df.copy()
    test_df = test_df.copy()

    # Exclude specified columns from scaling
    columns_to_scale = [col for col in train_df.columns if col not in columns_to_exclude]

    # Initialize the StandardScaler
    scaler = StandardScaler()

    # Fit and transform the training data
    train_data = scaler.fit_transform(train_df[columns_to_scale])

    # Transform the testing data using the same scaler
    test_data = scaler.transform(test_df[columns_to_scale])

    # Create new DataFrames with scaled data
    train_df[columns_to_scale] = train_data
    test_df[columns_to_scale] = test_data

    return train_df, test_df

    
    

In [ ]:
train_df, test_df = scale_dfs(train_df, test_df)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_df = train_df.sample(frac=0.4)

In [ ]:
train_df.to_parquet("stock_details_train.parquet")
test_df.to_parquet("stock_details_test.parquet")

In [ ]:
test_df[test_df['binary_target']==0]

In [ ]:
test_df[test_df['binary_target']==1]